# Level 0 vs Level 1: Model Comparison & Validation

**Purpose**: Compare baseline statistical classifier (Level 0) with neuro-symbolic approach (Level 1)

**Goals**:
- Send identical inputs to both models
- Compare predictions and confidence scores
- Identify ambiguity differences
- Highlight cases where rules change the decision
- Validate Level 1 improvements over Level 0

## Setup: Load Both Models

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load shared dataset
df = pd.read_csv('../data/intents_base.csv')
df['intent'] = df['intent'].str.lower().str.strip()

X = df['utterance']
y = df['intent']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Dataset loaded: {len(df)} records")
print(f"Train: {len(X_train)} | Test: {len(X_test)}")

## Level 0: Baseline Model Setup

In [ ]:
# Level 0 Configuration
LEVEL0_CONFIDENCE_THRESHOLD = 0.7

# Train Level 0 model
level0_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('classifier', LogisticRegression(
        solver='lbfgs',
        random_state=42,
        max_iter=1000
    ))
])

level0_pipeline.fit(X_train, y_train)
print("Level 0 model trained")

In [ ]:
# Level 0 Prediction Function
def level0_predict(text):
    """Level 0: TF-IDF + LR with confidence threshold"""
    pred_intent = level0_pipeline.predict([text])[0]
    proba = level0_pipeline.predict_proba([text])[0]
    confidence = float(np.max(proba))
    
    # Get probabilities for all classes
    classes = level0_pipeline.named_steps['classifier'].classes_
    proba_dict = {intent: float(proba[i]) for i, intent in enumerate(classes)}
    
    # Confidence thresholding
    final_intent = pred_intent if confidence >= LEVEL0_CONFIDENCE_THRESHOLD else 'abstain'
    
    return {
        'predicted_intent': pred_intent,
        'final_intent': final_intent,
        'confidence': confidence,
        'abstain': confidence < LEVEL0_CONFIDENCE_THRESHOLD,
        'probabilities': proba_dict
    }

print("Level 0 prediction function ready")

## Level 1: Neuro-Symbolic Model Setup

In [ ]:
# Level 1 Configuration
CONFIG = {
    'BASE_MIN_CONF': 0.60,
    'MIN_MARGIN': 0.10,
    'EXECUTION_MIN_CONF': 0.85,
    'MIN_TOKENS_OUT_OF_SCOPE': 3,
}

RULE_PRIORITY = ["R1", "R4", "R2", "R3"]

# Train Level 1 model (same statistical base as Level 0)
level1_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('classifier', LogisticRegression(
        solver='lbfgs',
        random_state=42,
        max_iter=1000
    ))
])

level1_pipeline.fit(X_train, y_train)
print("Level 1 model trained")
print(f"Rule Priority: {RULE_PRIORITY}")

In [ ]:
# Level 1 Signal Extraction
def extract_signals(text):
    """Extract signals from Level 1 model"""
    proba = level1_pipeline.predict_proba([text])[0]
    classes = level1_pipeline.named_steps['classifier'].classes_
    
    sorted_indices = np.argsort(proba)[::-1]
    max_confidence = float(proba[sorted_indices[0]])
    second_best_confidence = float(proba[sorted_indices[1]])
    margin = max_confidence - second_best_confidence
    
    # Get TF-IDF tokens
    tfidf_vec = level1_pipeline.named_steps['tfidf'].transform([text])
    active_features = tfidf_vec.toarray()[0]
    meaningful_tokens = np.sum(active_features > 0)
    
    return {
        'probabilities': {intent: float(proba[i]) for i, intent in enumerate(classes)},
        'max_confidence': max_confidence,
        'second_best_confidence': second_best_confidence,
        'margin': margin,
        'predicted_intent': classes[sorted_indices[0]],
        'meaningful_tokens': int(meaningful_tokens)
    }

print("Level 1 signal extraction ready")

In [ ]:
# Level 1 Decision Rules
def apply_decision_rules(signals):
    """Apply Level 1 neuro-symbolic rules"""
    triggered_rules = []
    predicted_intent = signals['predicted_intent']
    decision_state = 'accepted'
    decision_reason = 'model_prediction'
    
    # R1: Quality Gate
    if signals['meaningful_tokens'] < CONFIG['MIN_TOKENS_OUT_OF_SCOPE']:
        triggered_rules.append({
            'rule_id': 'R1',
            'category': 'quality',
            'priority': 100,
            'condition': f"meaningful_tokens < {CONFIG['MIN_TOKENS_OUT_OF_SCOPE']}",
            'signal_value': signals['meaningful_tokens']
        })
        decision_state = 'blocked'
        decision_reason = 'insufficient_tokens'
        return {
            'triggered_rules': triggered_rules,
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason
        }
    
    # R4: Safety Gate
    if signals['predicted_intent'] == 'execution' and signals['max_confidence'] < CONFIG['EXECUTION_MIN_CONF']:
        triggered_rules.append({
            'rule_id': 'R4',
            'category': 'safety',
            'priority': 90,
            'condition': f"predicted_intent==execution AND max_confidence < {CONFIG['EXECUTION_MIN_CONF']}",
            'signal_value': signals['max_confidence']
        })
        if signals['max_confidence'] >= CONFIG['BASE_MIN_CONF']:
            decision_state = 'blocked'
            decision_reason = 'execution_safety_block'
        else:
            decision_state = 'needs_clarification'
            decision_reason = 'execution_low_confidence'
        return {
            'triggered_rules': triggered_rules,
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason
        }
    
    # R2 & R3: Ambiguity Gates
    ambiguity_detected = False
    
    if signals['max_confidence'] < CONFIG['BASE_MIN_CONF']:
        triggered_rules.append({
            'rule_id': 'R2',
            'category': 'ambiguity',
            'priority': 50,
            'condition': f"max_confidence < {CONFIG['BASE_MIN_CONF']}",
            'signal_value': signals['max_confidence']
        })
        ambiguity_detected = True
    
    if signals['margin'] < CONFIG['MIN_MARGIN']:
        triggered_rules.append({
            'rule_id': 'R3',
            'category': 'ambiguity',
            'priority': 40,
            'condition': f"margin < {CONFIG['MIN_MARGIN']}",
            'signal_value': signals['margin']
        })
        ambiguity_detected = True
    
    if ambiguity_detected:
        decision_state = 'needs_clarification'
        decision_reason = 'ambiguous_prediction'
    
    return {
        'triggered_rules': triggered_rules,
        'predicted_intent': predicted_intent,
        'decision_state': decision_state,
        'decision_reason': decision_reason
    }

print("Level 1 decision rules ready")

In [ ]:
# Level 1 Prediction Function
def level1_predict(text):
    """Level 1: Statistical + Neuro-Symbolic Rules"""
    signals = extract_signals(text)
    rules_output = apply_decision_rules(signals)
    
    return {
        'predicted_intent': rules_output['predicted_intent'],
        'decision_state': rules_output['decision_state'],
        'decision_reason': rules_output['decision_reason'],
        'confidence': signals['max_confidence'],
        'margin': signals['margin'],
        'meaningful_tokens': signals['meaningful_tokens'],
        'triggered_rules': rules_output['triggered_rules'],
        'probabilities': signals['probabilities']
    }

print("Level 1 prediction function ready")

## Comparison Function

In [ ]:
def compare_predictions(utterance):
    """Compare Level 0 and Level 1 predictions side-by-side"""
    level0_result = level0_predict(utterance)
    level1_result = level1_predict(utterance)
    
    return {
        'utterance': utterance,
        'level0': level0_result,
        'level1': level1_result
    }

def display_comparison(comparison):
    """Display comparison in readable format"""
    print(f"\n{'='*100}")
    print(f"UTTERANCE: {comparison['utterance']}")
    print('='*100)
    
    l0 = comparison['level0']
    l1 = comparison['level1']
    
    # Side-by-side comparison
    print(f"\n{'LEVEL 0 (Baseline)':<50} | {'LEVEL 1 (Neuro-Symbolic)'}")
    print('-'*100)
    print(f"{'Predicted Intent: ' + l0['predicted_intent']:<50} | {'Predicted Intent: ' + l1['predicted_intent']}")
    print(f"{'Final Decision: ' + l0['final_intent']:<50} | {'Decision State: ' + l1['decision_state']}")
    print(f"{'Confidence: ' + f"{l0['confidence']:.4f}":<50} | {'Confidence: ' + f"{l1['confidence']:.4f}"}")
    print(f"{'Abstain: ' + str(l0['abstain']):<50} | {'Margin: ' + f"{l1['margin']:.4f}"}")
    print(f"{'—':<50} | {'Tokens: ' + str(l1['meaningful_tokens'])}")
    
    # Level 1 Rules
    if l1['triggered_rules']:
        print(f"{'—':<50} | {'Triggered Rules: ' + ', '.join([r['rule_id'] for r in l1['triggered_rules']])}")
        print(f"{'—':<50} | {'Decision Reason: ' + l1['decision_reason']}")
    else:
        print(f"{'—':<50} | {'Triggered Rules: None'}")
    
    # Ambiguity Detection
    print(f"\n{'-'*100}")
    print("AMBIGUITY ANALYSIS:")
    
    # Check for disagreement
    intent_match = l0['predicted_intent'] == l1['predicted_intent']
    print(f"  Intent Agreement: {'✓ YES' if intent_match else '✗ NO (DISAGREEMENT)'}")
    
    # Level 0 ambiguity (abstain)
    if l0['abstain']:
        print(f"  Level 0 Ambiguity: ⚠ ABSTAIN (confidence {l0['confidence']:.4f} < 0.70)")
    else:
        print(f"  Level 0 Ambiguity: ✓ Confident")
    
    # Level 1 ambiguity (rules)
    if l1['decision_state'] == 'needs_clarification':
        print(f"  Level 1 Ambiguity: ⚠ NEEDS CLARIFICATION ({l1['decision_reason']})")
    elif l1['decision_state'] == 'blocked':
        print(f"  Level 1 Ambiguity: 🚫 BLOCKED ({l1['decision_reason']})")
    else:
        print(f"  Level 1 Ambiguity: ✓ Accepted")
    
    # Probability distribution
    print(f"\n{'-'*100}")
    print("PROBABILITY DISTRIBUTION:")
    for intent in sorted(l0['probabilities'].keys()):
        l0_prob = l0['probabilities'][intent]
        l1_prob = l1['probabilities'][intent]
        diff = abs(l0_prob - l1_prob)
        match = "✓" if diff < 0.001 else "✗"
        print(f"  {intent:<20} L0: {l0_prob:.4f}  |  L1: {l1_prob:.4f}  {match}")
    
    print('='*100)

print("Comparison functions ready")

## Test Cases: Curated Scenarios

In [ ]:
# Test utterances covering different scenarios
test_utterances = [
    # High confidence cases
    "why is server cpu high",
    "summarize the incident from yesterday",
    
    # Execution cases (should trigger R4 in Level 1)
    "restart nginx on host123",
    "delete all logs from production",
    
    # Low token cases (should trigger R1 in Level 1)
    "hello",
    "hi",
    "ok",
    
    # Ambiguous cases
    "server issues",
    "check status",
    "what happened",
    
    # Edge cases
    "restart the server please",
    "tell me about server problems",
    "can you summarize",
]

print(f"Prepared {len(test_utterances)} test utterances")

## Run Comparison: All Test Cases

In [ ]:
# Compare all test cases
comparisons = []

for utterance in test_utterances:
    comparison = compare_predictions(utterance)
    comparisons.append(comparison)
    display_comparison(comparison)

## Summary: Disagreements & Ambiguity Detection

In [ ]:
# Analyze all comparisons
print("\n" + "="*100)
print("SUMMARY: Level 0 vs Level 1")
print("="*100)

disagreements = []
level0_abstains = []
level1_blocks = []
level1_clarifications = []
rule_triggers = {'R1': 0, 'R2': 0, 'R3': 0, 'R4': 0}

for comp in comparisons:
    l0 = comp['level0']
    l1 = comp['level1']
    utterance = comp['utterance']
    
    # Track disagreements
    if l0['predicted_intent'] != l1['predicted_intent']:
        disagreements.append({
            'utterance': utterance,
            'level0': l0['predicted_intent'],
            'level1': l1['predicted_intent']
        })
    
    # Track Level 0 abstains
    if l0['abstain']:
        level0_abstains.append(utterance)
    
    # Track Level 1 blocks
    if l1['decision_state'] == 'blocked':
        level1_blocks.append({
            'utterance': utterance,
            'reason': l1['decision_reason']
        })
    
    # Track Level 1 clarifications
    if l1['decision_state'] == 'needs_clarification':
        level1_clarifications.append({
            'utterance': utterance,
            'reason': l1['decision_reason']
        })
    
    # Track rule triggers
    for rule in l1['triggered_rules']:
        rule_triggers[rule['rule_id']] += 1

# Print summary
print(f"\nTotal Test Cases: {len(comparisons)}")
print(f"\nIntent Disagreements: {len(disagreements)}")
if disagreements:
    for d in disagreements:
        print(f"  • \"{d['utterance']}\" → L0: {d['level0']}, L1: {d['level1']}")

print(f"\nLevel 0 Abstains: {len(level0_abstains)}")
if level0_abstains:
    for utterance in level0_abstains:
        print(f"  • \"{utterance}\"")

print(f"\nLevel 1 Blocked: {len(level1_blocks)}")
if level1_blocks:
    for item in level1_blocks:
        print(f"  • \"{item['utterance']}\" → {item['reason']}")

print(f"\nLevel 1 Needs Clarification: {len(level1_clarifications)}")
if level1_clarifications:
    for item in level1_clarifications:
        print(f"  • \"{item['utterance']}\" → {item['reason']}")

print(f"\nLevel 1 Rule Triggers:")
for rule_id in RULE_PRIORITY:
    count = rule_triggers[rule_id]
    if count > 0:
        print(f"  {rule_id}: {count} times")

print("\n" + "="*100)

## Key Findings & Insights

In [ ]:
print("KEY INSIGHTS:")
print("="*100)

print("\n1. MODEL AGREEMENT:")
agreement_rate = (len(comparisons) - len(disagreements)) / len(comparisons) * 100
print(f"   - Intent agreement rate: {agreement_rate:.1f}%")
print(f"   - Both models use same statistical base (TF-IDF + LR)")
print(f"   - Disagreements indicate potential model instability")

print("\n2. AMBIGUITY HANDLING:")
print(f"   - Level 0: Uses fixed threshold (0.7), {len(level0_abstains)} abstains")
print(f"   - Level 1: Uses rules + thresholds, {len(level1_clarifications)} clarifications")
print(f"   - Level 1 adds execution safety: {len(level1_blocks)} blocked")

print("\n3. RULE EFFECTIVENESS:")
total_rule_triggers = sum(rule_triggers.values())
print(f"   - Total rule activations: {total_rule_triggers}")
print(f"   - Rules changed decision in {len(level1_blocks) + len(level1_clarifications)} cases")
print(f"   - Safety gates (R4) prevented risky executions: {rule_triggers['R4']} times")

print("\n4. LEVEL 1 IMPROVEMENTS:")
print("   ✓ Separates intent (what user wants) from decision (what system does)")
print("   ✓ Adds execution safety layer")
print("   ✓ Detects low-quality inputs (insufficient tokens)")
print("   ✓ Uses margin in addition to confidence")
print("   ✓ Provides structured, explainable decisions")

print("\n" + "="*100)

## Export Results to CSV

In [ ]:
# Create comparison dataframe
comparison_records = []

for comp in comparisons:
    l0 = comp['level0']
    l1 = comp['level1']
    
    comparison_records.append({
        'utterance': comp['utterance'],
        'l0_predicted_intent': l0['predicted_intent'],
        'l0_final_intent': l0['final_intent'],
        'l0_confidence': l0['confidence'],
        'l0_abstain': l0['abstain'],
        'l1_predicted_intent': l1['predicted_intent'],
        'l1_decision_state': l1['decision_state'],
        'l1_decision_reason': l1['decision_reason'],
        'l1_confidence': l1['confidence'],
        'l1_margin': l1['margin'],
        'l1_tokens': l1['meaningful_tokens'],
        'l1_rules_triggered': ', '.join([r['rule_id'] for r in l1['triggered_rules']]) if l1['triggered_rules'] else 'None',
        'intent_agreement': l0['predicted_intent'] == l1['predicted_intent']
    })

comparison_df = pd.DataFrame(comparison_records)

# Save to CSV
output_file = 'validation_results.csv'
comparison_df.to_csv(output_file, index=False)
print(f"Comparison results saved to: {output_file}")
print(f"\nPreview:")
comparison_df.head(10)